In [11]:
import os
import numpy as np
from skimage import io, filters, exposure, util

In [15]:
def gaussian_denoise_multipage_tiff(input_tiff, output_tiff, sigma=1.0):
    """
    Reads a multi-page TIFF into a 3D NumPy array, applies a Gaussian filter 
    to each Z-slice, rescales intensity, and saves the result as a multi-page TIFF.
    """
    # Read the multi-page TIFF as a 3D array: shape = (num_slices, height, width)
    image_stack = io.imread(input_tiff)
    print(f"Loaded stack with shape: {image_stack.shape} (Z, Y, X)")
    print(f"Original min/max: {image_stack.min()}, {image_stack.max()}")

    denoised_slices = []

    # Iterate through each Z-slice
    for i in range(image_stack.shape[0]):
        slice_i = image_stack[i, ...]
        
        # Apply Gaussian filter WITHOUT preserve_range
        denoised_slice_float = filters.gaussian(slice_i, sigma=sigma)

        # Optionally rescale intensities so they fit the slice's real min/max
        rescaled_slice = exposure.rescale_intensity(
            denoised_slice_float,
            in_range='image',   # auto-detect min/max of this slice
            out_range='uint16'  # you could also use 'dtype' if you prefer
        )

        # Now we have a rescaled 16-bit slice
        denoised_slices.append(rescaled_slice)

    # Stack denoised slices into a 3D NumPy array again
    denoised_stack = np.stack(denoised_slices, axis=0).astype(np.uint16)
    print(f"Denoised stack shape: {denoised_stack.shape} (Z, Y, X)")
    print(f"Denoised min/max: {denoised_stack.min()}, {denoised_stack.max()}")

    # Make sure the directory to output exists
    os.makedirs(os.path.dirname(output_tiff), exist_ok=True)

    # Save the denoised 3D stack as a multi-page TIFF
    io.imsave(output_tiff, denoised_stack)
    print(f"Saved denoised stack to: {output_tiff}")

if __name__ == "__main__":
    input_path = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/image_169-180/image_176_membrane_angle.tif"
    output_path = "/Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Denoised_Zstack_Images/image_176_membrane_angle_denoised_multipage.tif"
    sigma_value = 1.0

    gaussian_denoise_multipage_tiff(input_path, output_path, sigma=sigma_value)

Loaded stack with shape: (887, 817, 766) (Z, Y, X)
Original min/max: 0, 65535
Denoised stack shape: (887, 817, 766) (Z, Y, X)
Denoised min/max: 0, 65535
Saved denoised stack to: /Users/maryannjohnson/Library/CloudStorage/Box-Box/Documents/Fuse My Cells Challenge/Denoised_Zstack_Images/image_176_membrane_angle_denoised_multipage.tif
